In [1]:
import pandas as pd
import numpy as np
from ast import literal_eval
import requests
#from configs import api_key

c:\Users\Lales\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\Lales\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
c:\Users\Lales\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.XWYDX2IKJW2NMTWSFYNGFUWKQU3LYTCZ.gfortran-win_amd64.dll
  stacklevel=1)


# Extracting

### Extracting data from first Movielens dataset

In [2]:
movie_ml_25=pd.read_csv('../Resources/ml-25m/movies.csv')
movie_ml_25.head(2)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy


In [3]:
link_ml_25=pd.read_csv('../Resources/ml-25m/links.csv')
link_ml_25.head(2)

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0


In [4]:
link_ml_25.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62423 entries, 0 to 62422
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   movieId  62423 non-null  int64  
 1   imdbId   62423 non-null  int64  
 2   tmdbId   62316 non-null  float64
dtypes: float64(1), int64(2)
memory usage: 1.4 MB


### Merging dataframes

In [25]:
ml_25_df = pd.merge(link_ml_25, movie_ml_25, on="movieId")
ml_25_df.head(2)

,movieId,imdbId,tmdbId,title,genres
0,1,114709,862.0,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,113497,8844.0,Jumanji (1995),Adventure|Children|Fantasy


In [26]:
#extract year and title from title column
ml_25_df = ml_25_df.assign(year = lambda x: x['title'].str[-5:-1])
ml_25_df = ml_25_df.assign(title = lambda x: x['title'].str[:-7])
ml_25_df.head(2)

,movieId,imdbId,tmdbId,title,genres,year
0,1,114709,862.0,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,113497,8844.0,Jumanji,Adventure|Children|Fantasy,1995


In [28]:
ml_25_df.drop(['tmdbId'],axis=1,inplace=True)
ml_25_df.isna().sum()

movieId    0
imdbId     0
title      0
genres     0
year       0
dtype: int64

In [29]:
len(ml_25_df)

62423

### Extracting data from metadata Movielens json 

In [31]:
json_metadata=pd.read_json('../Resources/movie_dataset_public_final/raw/metadata_updated.json', lines=True)
json_metadata.head(2)

,title,directedBy,starring,avgRating,imdbId,item_id
0,Toy Story (1995),John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,114709,1
1,Jumanji (1995),Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,113497,2


In [32]:
#extract year and title from title column
json_metadata = json_metadata.assign(year = lambda x: x['title'].str[-5:-1])
json_metadata = json_metadata.assign(title = lambda x: x['title'].str[:-7])
json_metadata.head(2)

,title,directedBy,starring,avgRating,imdbId,item_id,year
0,Toy Story,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146,114709,1,1995
1,Jumanji,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605,113497,2,1995


In [33]:
#checking null values after extracting
json_metadata.isna().sum()

title         0
directedBy    0
starring      0
avgRating     0
imdbId        0
item_id       0
year          0
dtype: int64

### Merging Movielense dataframes 

In [38]:
df=json_metadata[['imdbId','directedBy','starring','avgRating']]
mvielense_df = pd.merge(ml_25_df, df, on=["imdbId"],how='inner')
mvielense_df.head(2)

,movieId,imdbId,title,genres,year,directedBy,starring,avgRating
0,1,114709,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995,John Lasseter,"Tim Allen, Tom Hanks, Don Rickles, Jim Varney,...",3.89146
1,2,113497,Jumanji,Adventure|Children|Fantasy,1995,Joe Johnston,"Jonathan Hyde, Bradley Pierce, Robin Williams,...",3.26605


In [39]:
len(mvielense_df)

62417

In [40]:
#checking null values after extracting
mvielense_df.isna().sum()

movieId       0
imdbId        0
title         0
genres        0
year          0
directedBy    0
starring      0
avgRating     0
dtype: int64

In [53]:
mvielense_df.to_csv('output_files/movie.csv',index=False)

## Extracting users rating files

In [43]:
rating_df=pd.read_csv('../Resources/ml-25m/ratings.csv')
rating_df.head(2)

,userId,movieId,rating,timestamp
0,1,296,5.0,1147880044
1,1,306,3.5,1147868817


In [44]:
tag_df=pd.read_csv('../Resources/ml-25m/tags.csv')
tag_df.head(2)

,userId,movieId,tag,timestamp
0,3,260,classic,1439472355
1,3,260,sci-fi,1439472256


In [45]:
#checking null value
rating_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

# Transforming to MongoDB

In [46]:
from pymongo import MongoClient 
client = MongoClient() 
db=client.Movielens

In [48]:
movie_dict=mvielense_df.to_dict(orient = 'records')
rate_dict=rating_df.to_dict(orient = 'records')

In [49]:
db.rating.drop()
result1 = db.rating.insert_many(rate_dict)

In [50]:
db.movie.drop()
result2 = db.movie.insert_many(movie_dict)

# Loading

In [52]:
client = MongoClient() 
db=client.Movielens
listings = db.movie.find_one()
print(listings)

{'_id': ObjectId('628a4038188c64eb8c4a565f'), 'movieId': 1, 'imdbId': 114709, 'title': 'Toy Story', 'genres': 'Adventure|Animation|Children|Comedy|Fantasy', 'year': '1995', 'directedBy': 'John Lasseter', 'starring': 'Tim Allen, Tom Hanks, Don Rickles, Jim Varney, John Ratzenberger, Wallace Shawn, Laurie Metcalf, John Morris, R. Lee Ermey, Annie Potts', 'avgRating': 3.89146}
